# Time Travelling

Another advantage of Iceberg's metadata structure is that it gives us Time Travel for free. Since all we're doing is storing snapshots and moving pointers, time travelling is essentially just asking to see the data at a previous pointer. 

In [1]:
import sqlalchemy as sa
import polars as pl

In [2]:
engine = sa.create_engine("trino://trino:@trino:8080/lakekeeper")

In [6]:
with engine.connect() as conn:
    df = pl.read_database('SELECT * FROM house_prices."raw$snapshots"', conn)
df    

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
"datetime[μs, UTC]",i64,i64,str,str,struct[21]
2025-04-30 20:26:57.891 UTC,1092067571415845178,7243334300264100548,"""overwrite""","""s3://warehouse/house_prices/ra…","{null,""841772"",""0"",null,""0"",""1"",null,null,null,null,null,null,null,null,""19093353"",null,null,""0"",""15838359"",""1"",""704344""}"
2025-04-30 19:43:41.559 UTC,3870307059191542653,null,"""append""","""s3://warehouse/house_prices/ra…","{""15838359"",""704344"",""0"",null,""0"",""1"",""1"",""704344"",null,null,null,null,null,null,""15838359"",null,null,""0"",null,null,null}"
2025-04-30 20:18:01.857 UTC,7243334300264100548,4996382859333512033,"""append""","""s3://warehouse/house_prices/ra…","{null,""1546116"",""0"",null,""0"",""2"",null,null,null,null,null,null,null,null,""34931712"",null,null,""0"",null,null,null}"
2025-04-30 20:09:37.002 UTC,122416445074455388,6160762292259832203,"""append""","""s3://warehouse/house_prices/ra…","{null,""1546116"",""0"",null,""0"",""2"",null,null,null,null,null,null,null,null,""34931712"",null,null,""0"",null,null,null}"
2025-05-01 19:17:21.296 UTC,6910125867099050337,5618685459824118337,"""overwrite""","""s3://warehouse/house_prices/ra…","{""19156388"",""2387888"",""841772"",""1"",""1"",""3"",""1"",""841772"",""Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)"",""trino"",""20250501_191715_00006_hpuqw"",""841772"",""475"",""trino"",""54192605"",""1"",""1"",""0"",null,null,null}"
2025-04-30 20:26:58.422 UTC,5618685459824118337,1092067571415845178,"""append""","""s3://warehouse/house_prices/ra…","{""15942864"",""1546116"",""0"",null,""0"",""2"",""1"",""704344"",null,null,null,null,null,null,""35036217"",null,null,""0"",null,null,null}"
2025-04-30 20:09:42.346 UTC,4389943633381866277,122416445074455388,"""append""","""s3://warehouse/house_prices/ra…","{null,""1546116"",""0"",null,""0"",""2"",null,null,null,null,null,null,null,null,""34931712"",null,null,""0"",null,null,null}"
2025-04-30 20:17:56.555 UTC,4996382859333512033,4389943633381866277,"""append""","""s3://warehouse/house_prices/ra…","{null,""1546116"",""0"",null,""0"",""2"",null,null,null,null,null,null,null,null,""34931712"",null,null,""0"",null,null,null}"
2025-04-30 19:46:09.052 UTC,6160762292259832203,3870307059191542653,"""append""","""s3://warehouse/house_prices/ra…","{""19093353"",""1546116"",""0"",null,""0"",""2"",""1"",""841772"",null,null,null,null,null,null,""34931712"",null,null,""0"",null,null,null}"
